In [ ]:
import time
import urllib.parse
import hashlib
import hmac
import base64
import requests
import threading

class Kraken(object):

    nonce_lock = threading.Lock()

    def get_kraken_signature(self, urlpath, data):
        postdata = urllib.parse.urlencode(data)
        encoded = (str(data['nonce']) + postdata).encode()
        message = urlpath.encode() + hashlib.sha256(encoded).digest()

        mac = hmac.new(base64.b64decode(self.api_sec), message, hashlib.sha512)
        sigdigest = base64.b64encode(mac.digest())
        return sigdigest.decode()

    def kraken_request(self, uri_path, data):
        headers = {}
        headers['API-Key'] = self.api_key
        headers['API-Sign'] = self.get_kraken_signature(uri_path, data)
        req = requests.post((self.api_url + uri_path), headers=headers, data=data)
        print(f"응답 코드: {req.status_code}")
        return req

    def get_nonce(self):
        with self.nonce_lock:
            # millisecond 단위로 시간을 기반으로 고유한 nonce 생성
            nonce = str(int(time.time() * 1000))
            time.sleep(0.001)  # 너무 빠른 중복 방지를 위해 짧은 대기 추가
            return nonce

    def get_balance(self):
        # 스레드마다 고유한 nonce 생성
        nonce = self.get_nonce()
        resp = self.kraken_request('/0/private/Balance', {
            "nonce": nonce
        })
        return resp.json()

def request_balance(kraken):
    while True:
        balance_response = kraken.get_balance()
        print(balance_response)
        if 'error' in balance_response and any("EAPI:Rate limit exceeded" in error for error in balance_response['error']):
            print("API 호출 한도를 초과했습니다!")
            break
        time.sleep(0.01)  # 요청 간 적당한 대기 시간

if __name__ == '__main__':
    kr = Kraken()
    
    # 15개의 스레드 생성
    threads = []
    for _ in range(15):
        t = threading.Thread(target=request_balance, args=(kr,))
        threads.append(t)
        t.start()

    # 모든 스레드가 종료될 때까지 대기
    for t in threads:
        t.join()
